In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
data = pd.read_csv('mov1.csv')
data.columns

Index(['adulto', 'orçamento', 'idioma_original', 'título_original',
       'visão_geral', 'popularidade', 'receita', 'tempo_de_execução',
       'voto_médio', 'contagem_de_votos', 'palavras_chave', 'gênero1',
       'gênero2', 'gênero3', 'elenco', 'diretor', 'ano_lançamento',
       'mês_lançamento', 'dia_lançamento', 'dia_semana_lançamento',
       'empresa_de_produção', 'país_de_produção', 'lucro_filme',
       'lucro_porcentagem'],
      dtype='object')

In [3]:
# Filtrar linhas onde 'voto_médio' é maior que 0 e 'contagem_de_votos' é maior que 20
data_filtrado = data[(data['voto_médio'] > 0) & (data['contagem_de_votos'] > 20) & (data['receita'] > 0) & (data['tempo_de_execução'] > 50) & (data['popularidade'] < 300) & (data['orçamento'] > 1000) &  (data['lucro_porcentagem'] < 100000)]

# Lista de todas as coluna1s numéricas do DataFrame
colunas_numericas1 = ['orçamento', 'popularidade', 'receita', 'tempo_de_execução', 'voto_médio']


# Criar gráficos para cada coluna1 numérica em comparação com 'voto_médio'
figuras1 = []
for coluna1 in colunas_numericas1:
    if coluna1 != 'voto_médio':  # Evitar comparar 'voto_médio' com ele mesmo
        fig1 = px.scatter(data_filtrado, x=coluna1, y='voto_médio', 
                         title=f'{coluna1.capitalize()} vs. Voto Médio (Votos > 20, Recta > 0, Execução > 50, Plde < 300, Orçamento > 1000, Lucro em % < 100000)',
                         labels={coluna1: coluna1.capitalize(), 'voto_médio': 'Voto Médio'})
        figuras1.append(fig1)

# Mostrar gráficos
for fig1 in figuras1:
    fig1.show()


In [4]:
colunas_numericas2 = ['contagem_de_votos', 'ano_lançamento', 'mês_lançamento', 'dia_lançamento', 'lucro_filme', 'lucro_porcentagem']

# Criar gráficos para cada coluna2 numérica em comparação com 'voto_médio'
figuras2 = []
for coluna2 in colunas_numericas2:
    if coluna2 != 'voto_médio':  # Evitar comparar 'voto_médio' com ele mesmo
        if coluna2 in ['ano_lançamento', 'mês_lançamento', 'dia_lançamento']:
            # Calcular os valores do boxplot
            medianas = data_filtrado.groupby(coluna2)['voto_médio'].median().reset_index()
            medianas_sorted = medianas.sort_values('voto_médio', ascending=False)
            n = len(medianas_sorted)
            # Escolher uma paleta de cores baseada na escala de plasma do Plotly
            color_scale = px.colors.sequential.Plasma[::-1]
            # Ajustar o tamanho da lista de cores se for menor que o número de categorias
            if n > len(color_scale):
                color_scale *= (n // len(color_scale)) + 1
            # Mapear as cores de acordo com a ordem das medianas
            color_map = {medianas_sorted.iloc[i][coluna2]: color_scale[i] for i in range(n)}
            
            # Criar boxplot para as colunas 'ano_lançamento', 'mês_lançamento' e 'dia_lançamento'
            fig2 = px.box(data_filtrado, x=coluna2, y='voto_médio', color=coluna2, color_discrete_map=color_map,
                          title=f'{coluna2.capitalize()} vs. Voto Médio (Contagem de Votos > 20, Receita > 0, Tempo de execução > 50, Popularidade < 300. Orçamento > 1000 )',
                          labels={coluna2: coluna2.capitalize(), 'voto_médio': 'Voto Médio'})
        else:
            # Criar scatter plot para as outras colunas numéricas
            fig2 = px.scatter(data_filtrado, x=coluna2, y='voto_médio', 
                              title=f'{coluna2.capitalize()} vs. Voto Médio (Contagem de Votos > 20, Receita > 0, Tempo de execução > 50, Popularidade < 300. Orçamento > 1000 )',
                              labels={coluna2: coluna2.capitalize(), 'voto_médio': 'Voto Médio'})
        figuras2.append(fig2)

# Mostrar gráficos
for fig2 in figuras2:
    fig2.show()

In [5]:
# Definir faixas de orçamento
faixas_orcamento = [(0, 3000000), (3000000, 40000000), (40000000, 100000000), (100000000, float('inf'))]
titulos = ['até 3 milhões', 'entre 3 e 40 milhões', 'entre 40 e 100 milhões', 'acima de 100 milhões']

# Criar e mostrar os gráficos para cada faixa de orçamento
for faixa, titulo in zip(faixas_orcamento, titulos):
    data_faixa = data_filtrado[(data_filtrado['orçamento'] > faixa[0]) & (data_filtrado['orçamento'] <= faixa[1])]
    fig_orcamento = px.scatter(data_faixa, x='orçamento', y='voto_médio', title=f'Voto Médio vs Orçamento {titulo}')
    fig_orcamento.show()


In [6]:
# Definir os intervalos manualmente
intervals = [0, 3000000, 40000000, 100000000, np.inf]

# Gráficos para orçamento filtrados
orcamentos = [data_filtrado[(data_filtrado['orçamento'] > intervals[i]) & (data_filtrado['orçamento'] <= intervals[i+1])] for i in range(len(intervals)-1)]

# Função para agrupar dados em intervalos
def group_data(data, intervals):
    labels = [f'Intervalo {i+1}' for i in range(len(intervals)-1)]
    grouped_data = pd.cut(data, bins=intervals, labels=labels, include_lowest=True)
    return grouped_data

# Número de intervalos para agrupar os dados
num_intervals = len(intervals) - 1

# Agrupar dados de orçamento em intervalos
data_filtrado['orçamento_agrupado'] = group_data(data_filtrado['orçamento'], intervals)

# Plotar boxplot para orçamento agrupado
fig_orcamento_boxplot = px.box(data_filtrado, x='orçamento_agrupado', y='voto_médio', title='Voto Médio por Intervalo de Orçamento')

# Outros tipos de gráficos
fig_popularidade_histogram = px.histogram(data_filtrado, x='popularidade', title='Histograma de Popularidade')
fig_receita_densidade = px.density_contour(data_filtrado, x='receita', y='voto_médio', title='Densidade de Voto Médio em função da Receita')

# Mostrar gráficos
fig_orcamento_boxplot.show()
fig_popularidade_histogram.show()
fig_receita_densidade.show()


C:\Users\psyte\AppData\Local\Temp\ipykernel_52924\3299634084.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
# Função para agrupar dados em intervalos
group_data = lambda data, intervals: pd.cut(data, bins=intervals, labels=[f'Intervalo {i+1}' for i in range(len(intervals)-1)], include_lowest=True)

# Número de intervalos para agrupar os dados
num_intervals = len(intervals) - 1

# Agrupar dados de orçamento em intervalos
data_filtrado['orçamento_agrupado'] = group_data(data_filtrado['orçamento'], intervals)

# Plotar boxplot e gráficos de densidade e histograma
fig_orcamento_densidade = [px.density_contour(orc, x='orçamento', y='voto_médio', title=f'Densidade de Voto Médio para Orçamento {"até 3 milhões" if i==0 else f"entre {intervals[i]} e {intervals[i+1]}"}') for i, orc in enumerate(orcamentos)]
fig_orcamento_histograma = [px.histogram(orc, x='voto_médio', title=f'Histograma de Voto Médio para Orçamento {"até 3 milhões" if i==0 else f"entre {intervals[i]} e {intervals[i+1]}"}') for i, orc in enumerate(orcamentos)]

# Mostrar gráficos

[fig.show() for fig in fig_orcamento_densidade]
[fig.show() for fig in fig_orcamento_histograma]


C:\Users\psyte\AppData\Local\Temp\ipykernel_52924\3969085800.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[None, None, None, None]

# Análise dos Insights Obtidos dos Gráficos

## 1. Popularidade e Voto Médio:
   - A popularidade do filme, quando muito alta, tem consistentemente um voto médio maior que 6. No entanto, uma popularidade baixa não significa necessariamente que o voto médio do filme será baixo. Filmes com popularidade menor também podem ter um voto médio maior. É interessante notar que filmes com menores votos médios tendem a ter também popularidades menores.

## 2. Orçamento e Voto Médio:
   - O orçamento tem uma influência relativamente pequena no voto médio dos filmes. Os filmes com menores votos médios estão concentrados em orçamentos baixos. No entanto, há filmes com orçamentos muito baixos que têm um voto médio maior do que filmes com orçamentos muito altos.

## 3. Receita e Voto Médio:
   - Receitas altas dos filmes geralmente indicam que o filme teve um voto médio acima da média, quase sempre acima de 5.5. Os filmes com menores votos médios são os de receita mais baixa. No entanto, há filmes com receitas muito menores que outros e com votos médios significativamente mais altos.

## 4. Tempo de Execução e Voto Médio:
   - Filmes com mais de 150 minutos têm quase sempre um voto médio mais alto, acima de 6. Embora haja uma tendência de filmes mais longos terem maiores votos médios, existem filmes com menos de 90 minutos e com voto médio muito alto. Os menores votos médios estão concentrados em filmes entre 80 e 121 minutos. No entanto, não é possível afirmar que filmes com duração nesse intervalo terão voto médio pequeno, pois existem muitos filmes nesse intervalo com voto médio maior que 7.

## 5. Contagem de Votos e Voto Médio:
   - Filmes com muitos votos tendem a ter um voto médio maior que 6, acima da média. Os filmes com menores votos médios são aqueles com menor quantidade de votos. No entanto, existem filmes com pequena quantidade de votos e voto médio alto.

## 6. Ano de Lançamento e Voto Médio:
   - É possível notar uma tendência do voto médio diminuir ao longo do tempo. Filmes mais antigos tendem a ter voto médio maiores que filmes lançados mais recentemente. A partir de 1977 já é possível notar que o voto médio dos filmes foi diminuindo. No entanto, de 1977 em diante não é possível dizer que o voto médio continuou caindo, ele parece continuar o mesmo, com alguns anos aumentando um pouco e outros anos diminuindo um pouco.

## 7. Mês de Lançamento e Voto Médio:
   - A diferença não é muito grande sobre os votos médios em relação ao mês de lançamento. No entanto, é possível ver um ligeiro voto médio maior para filmes lançados no fim do ano, em comparação com filmes lançados de janeiro até junho.

## 8. Dia do Lançamento e Voto Médio:
   - O dia do lançamento não parece ter uma influência significativa no voto médio do filme.

## 9. Lucro do Filme e Voto Médio:
   - Filmes com lucros muito altos tendem a ter um voto médio mais alto. No entanto, existem filmes com baixo lucro e voto médio alto. Os filmes com menores votos médios são geralmente os que têm menores lucros, ou até mesmo lucros negativos. No entanto, existem alguns filmes com lucro até mesmo negativo e com voto médio maior que 7.

## 10. Lucro em Porcentagem e Voto Médio:
   - Filmes com lucro em porcentagem muito altos tendem a ter voto médio mais alto. No entanto, existem filmes com pequeno lucro em porcentagem e com voto médio muito alto. Inclusive, o maior voto médio é de um filme com 657% em lucro em porcentagem. É possível ver inúmeros filmes com lucro maior que esse filme e voto médio menor. Os piores votos médios são de filmes com pequenos lucros em porcentagem.

## 11. Relação entre Voto Médio e Orçamento:
   - No filtro de voto médio versus orçamento até 3 milhões, não é possível observar uma relação clara, com pontos do scatter plot sendo distribuídos quase homogeneamente.

## 12. Relação entre Orçamento e Voto Médio:
   - Isso também se aplica para os filtros de orçamento versus voto médio.

Esses insights são importantes para entendermos melhor como diferentes variáveis influenciam o voto médio dos filmes, permitindo uma análise mais abrangente e precisa.

# Conclusão

Após analisar os diversos insights obtidos dos gráficos relacionados ao voto médio dos filmes, algumas conclusões podem ser destacadas:

1. **Diversidade de Fatores Influenciando o Voto Médio:** Foi evidente que uma variedade de fatores, como popularidade, orçamento, receita, tempo de execução, entre outros, podem influenciar o voto médio dos filmes. No entanto, a magnitude e a direção dessa influência variam de acordo com cada variável, mostrando a complexidade por trás da determinação do sucesso de um filme.

2. **Correlação Nem Sempre Implica Causalidade:** Embora certas variáveis tenham mostrado correlação com o voto médio, como lucro alto ou popularidade elevada, é importante ressaltar que correlação não implica necessariamente causalidade. Ou seja, apenas porque duas variáveis estão correlacionadas, não significa que uma causa a outra.

3. **Variações e Exceções:** Houve casos de filmes com características atípicas, como baixo orçamento e alto voto médio, ou filmes com alta popularidade, mas votos médios abaixo da média. Isso ressalta a importância de considerar cada filme individualmente e reconhecer que existem exceções às tendências gerais.

4. **Complexidade ao Longo do Tempo:** A análise temporal mostrou que o voto médio dos filmes parece ter diminuído ao longo das décadas, mas com flutuações em períodos mais recentes. Essa complexidade sugere que outros fatores, além do tempo, podem estar influenciando as preferências do público.

5. **Necessidade de Abordagem Multifacetada:** Diante da diversidade de fatores influenciadores, uma abordagem multifacetada é necessária para entender completamente o comportamento do voto médio dos filmes. Considerar não apenas uma, mas várias variáveis em conjunto, pode proporcionar uma visão mais abrangente e precisa.

Em suma, a análise dos gráficos ofereceu insights valiosos sobre os determinantes do voto médio dos filmes, destacando a complexidade e a multidimensionalidade desse fenômeno. Essas descobertas são cruciais para profissionais da indústria cinematográfica e pesquisadores interessados em compreender e prever o sucesso dos filmes.


In [8]:
def agrupar_categorias(df, coluna, limite=10):
    df = df.copy()  # Faz uma cópia do DataFrame original
    contagem = df[coluna].value_counts()
    categorias_principais = contagem[:limite].index.tolist()
    df.loc[~df[coluna].isin(categorias_principais), coluna] = 'Outros'
    return df

# Criar a coluna 'todos_gêneros' combinando as informações das colunas 'genero1', 'genero2' e 'genero3'
data_filtrado['todos_gêneros'] = data_filtrado[['gênero1', 'gênero2', 'gênero3']].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Agrupar categorias menos frequentes para diretor, país de produção, idioma original, empresa de produção e gênero
categorias = ['diretor', 'país_de_produção', 'idioma_original', 'empresa_de_produção']
for categoria in categorias + ['todos_gêneros']:
    data_filtrado = agrupar_categorias(data_filtrado, categoria)

# Desagregar a lista 'todos_gêneros' para criar uma nova linha para cada gênero
data_filtrado_generos = data_filtrado.explode('todos_gêneros')

# Criar o boxplot para cada categoria
figuras = []
for categoria in categorias + ['todos_gêneros']:
    fig = px.box(data_filtrado_generos, x=categoria, y='voto_médio', color=categoria, title=f'Boxplot do Voto Médio por {categoria.capitalize()}')
    fig.update_xaxes(categoryorder='total descending')
    figuras.append(fig)

# Mostrar os gráficos
for figura in figuras:
    figura.show()



C:\Users\psyte\AppData\Local\Temp\ipykernel_52924\1900825846.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
import os
import plotly.io as pio

# Pasta de destino para os gráficos
pasta_destino = "graficos_votomedio"

# Verificar e criar a pasta de destino, se necessário
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

# Lista para armazenar o HTML de cada gráfico
html_graficos = []

# Salvar as figuras como HTML na pasta de destino
for i, figura in enumerate(figuras1 + figuras2, start=1):
    nome_arquivo = f"{pasta_destino}/figura_{i}.html"
    pio.write_html(figura, nome_arquivo)
    html_graficos.append(pio.to_html(figura, full_html=False))

# Salvar os gráficos de faixa de orçamento como HTML na pasta de destino
for i, (fig_orcamento_dens, fig_orcamento_hist) in enumerate(zip(fig_orcamento_densidade, fig_orcamento_histograma), start=len(figuras1 + figuras2) + 1):
    nome_arquivo_dens = f"{pasta_destino}/figura_{i}_densidade.html"
    nome_arquivo_hist = f"{pasta_destino}/figura_{i}_histograma.html"
    pio.write_html(fig_orcamento_dens, nome_arquivo_dens)
    pio.write_html(fig_orcamento_hist, nome_arquivo_hist)
    html_graficos.append(pio.to_html(fig_orcamento_dens, full_html=False))
    html_graficos.append(pio.to_html(fig_orcamento_hist, full_html=False))

# Salvar os boxplots como HTML na pasta de destino
for i, figura in enumerate(figuras, start=len(figuras1 + figuras2) + len(fig_orcamento_densidade) + len(fig_orcamento_histograma) + 1):
    nome_arquivo = f"{pasta_destino}/figura_{i}.html"
    pio.write_html(figura, nome_arquivo)
    html_graficos.append(pio.to_html(figura, full_html=False))

# Combine o HTML de todos os gráficos em uma única string
html_completo = "\n".join(html_graficos)

# Salvar o arquivo HTML com utf-8
nome_arquivo_html = f"{pasta_destino}/graficos_votomedio.html"
with open(nome_arquivo_html, "w", encoding="utf-8") as f:
    f.write(html_completo)

print(f"Gráficos salvos com sucesso em '{nome_arquivo_html}'.")


Gráficos salvos com sucesso em 'graficos_votomedio/graficos_votomedio.html'.


## Análise Detalhada dos Votos Médios dos Filmes (Parte 2) - 

### Voto Médio por Diretor
**Observações:**
- **Diversidade de Nacionalidades:** A análise revela uma diversidade de nacionalidades entre os diretores com votos médios mais altos, com diretores americanos como Woody Allen e Steven Spielberg, além de diretores de outros países como Clint Eastwood (Reino Unido), Dean Semler (Austrália) e Roger Deakins (Inglaterra). Essa diversidade indica que a qualidade percebida dos filmes não se limita a uma única nacionalidade.
- **Padrões de Nacionalidade:** No entanto, vale ressaltar que a maioria dos diretores na lista dos mais votados (6 dos 10) são americanos. Isso sugere uma possível tendência de que diretores americanos tenham uma presença mais significativa entre os votos médios mais altos.
- **Fatores Influenciadores:** A diversidade de nacionalidades entre os diretores com votos médios altos indica que o sucesso de um filme não depende apenas da nacionalidade do diretor. Outros fatores como estilo cinematográfico, filmografia, aclamação da crítica e apelo universal do filme também podem ter um papel importante.

### Voto Médio por País de Produção
**Observações:**
- **Domínio dos EUA e Reino Unido:** Os Estados Unidos e o Reino Unido se destacam como os países com os votos médios mais altos, reforçando sua influência dominante na indústria cinematográfica global.
- **Presença de Outros Países:** No entanto, a presença de países como França, Alemanha, Canadá, Índia, Austrália, Itália e Japão entre os 10 países com votos médios mais altos indica que a produção cinematográfica de qualidade não se limita exclusivamente aos EUA e Reino Unido.
- **Fatores Influenciadores:** O sucesso de filmes de diferentes países pode ser explicado por diversos fatores, como cultura cinematográfica, orçamento, elenco internacional, temas universais e estratégias de marketing específicas para cada mercado.

### Voto Médio por Idioma
**Observações:**
- **Predominância do Inglês:** O inglês se destaca como o idioma com os votos médios mais altos, seguido pelo francês. Isso pode ser explicado pela ampla distribuição de filmes em inglês e pela familiaridade do público global com esse idioma.
- **Presença de Outros Idiomas:** No entanto, a presença de idiomas como hindi, espanhol, italiano, chinês, japonês, alemão e russo entre os 10 idiomas com votos médios mais altos indica que o público também aprecia filmes em outros idiomas, especialmente quando se conectam com temas universais ou possuem elementos culturais relevantes.
- **Fatores Influenciadores:** O sucesso de filmes em diferentes idiomas pode ser explicado por diversos fatores, como familiaridade do público com o idioma, dublagem de qualidade, temas universais, apelo cultural e estratégias de marketing específicas para cada idioma.


### Voto Médio por Empresa de Produção

**Observações:**
- **Diversidade de Empresas:** A análise revela uma grande diversidade de empresas de produção entre as com votos médios mais altos, com empresas americanas como Universal Pictures, Paramount Pictures e Warner Bros., além de empresas de outros países como Twentieth Century Fox (EUA e Reino Unido), Columbia Pictures (EUA), New Line Cinema (EUA), United Artists (EUA), Miramax Films (EUA), Walt Disney (EUA) e empresas de outros países como Italia (Mediaset) e França (Gaumont). Essa diversidade indica que a qualidade percebida dos filmes não se limita a um único país ou empresa.
- **Padrões de Nacionalidade:** No entanto, vale ressaltar que a maioria das empresas na lista das mais votadas (8 das 10) são americanas. Isso sugere uma possível tendência de que empresas de produção americanas tenham uma presença mais significativa entre os votos médios mais altos.
- **Fatores Influenciadores:** O sucesso de filmes de diferentes empresas de produção pode ser explicado por diversos fatores, como reputação da empresa, gênero dos filmes, estratégias de marketing e colaborações com diretores e talentos de renome.



### Voto Médio por Gênero
**Observações:**
- **Predominância do Drama:** O drama se destaca como o gênero com o voto médio mais alto, seguido pela comédia. Isso pode ser explicado pelo fato de que o drama permite explorar uma ampla gama de temas e emoções humanas, o que pode ressoar com um público amplo. A comédia, por sua vez, também pode alcançar um público vasto devido ao seu potencial de entretenimento e leveza.
- **Presença de Outros Gêneros:** A presença de gêneros como suspense, terror, ação, romance e animação entre os 10 gêneros com votos médios mais altos indica que o público aprecia uma variedade de estilos cinematográficos.
- **Fatores Influenciadores:** O sucesso de filmes de diferentes gêneros pode ser explicado por diversos fatores, como preferência do público, complexidade narrativa, estilo cinematográfico, qualidade da produção e elenco.


### **Análise Detalhada:**
- Uma análise mais aprofundada dos filmes de cada diretor, país de produção, idioma, empresa de produção e gênero, considerando diversos fatores como nacionalidade, reputação, gênero dos filmes produzidos, estratégias de marketing e outras variáveis relevantes, pode fornecer insights mais precisos sobre as características que contribuem para seus respectivos votos médios.
